# Практика
Используемые библиотеки

In [165]:
import pandas as pd
import numpy as np
import requests
import io
import re
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [166]:
"""url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))"""
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [167]:
df["must_between"] = ['yes' if x else 'no' for x in df['Норматив'].str.contains("в пределах")]
df['val'] = df['Норматив'].str.replace('не более ', '')
df['val'] = df['val'].str.replace('в пределах', '')
df['Результат анализа'] = df['Результат анализа'].str.replace('б/цвета', '0')
df = df.join(df['val'].str.split('-', expand=True))
df = df.rename(columns={0: 'first_limit', 1: 'second_limit'})
df['Результат анализа'] = df['Результат анализа'].astype(float)
df['first_limit'] = df['first_limit'].str.replace(',', '.')
df['second_limit'] = df['second_limit'].str.replace(',', '.')
df['first_limit'] = df['first_limit'].astype(float)
df['second_limit'] = df['second_limit'].astype(float)
del df['val']
df = df.set_index('Показатель')



In [168]:
def myfunc(must_between, first_limit, second_limit, val):
     if (must_between == 'yes' and val > first_limit and val < second_limit):
        return 'OK'
     elif must_between == 'no' and val < first_limit:
        return 'OK'
     else:
        return 'Bad'

df['rez'] = df.apply(lambda x: myfunc(x['must_between'], x['first_limit'], x['second_limit'], x['Результат анализа']), axis=1)

df

,Единица измерений,Результат анализа,Норматив,must_between,first_limit,second_limit,rez
Показатель,,,,,,,
pH,единицы pH,8.400,в пределах 6-9,yes,6.0,9.0,OK
Запах,баллы,1.000,не более 2-3,no,2.0,3.0,OK
Цветность,градусы,0.000,не более 30,no,30.0,NaN,OK
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,yes,7.0,10.0,OK
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",no,1.5,NaN,OK
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",no,3.3,NaN,OK
Нитраты (по NO3),мг/дм3,24.000,не более 45,no,45.0,NaN,OK
Фосфаты (P),мг/дм3,0.360,"не более 3,5",no,3.5,NaN,OK
Хлориды (Cl),мг/дм3,200.000,не более 350,no,350.0,NaN,OK


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [169]:
n = 10000000
box = ['Апельсин', 'Яблоко', 'Апельсин', 'Апельсин','Апельсин', 'Яблоко', 'Яблоко', 'Яблоко', 'Апельсин']
r=0
for i in range(n):
    sample = random.sample(box,3)
    if sample.count('Апельсин') == 3:
        r+=1
r/n

0.1188209

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [170]:
n=100000
box = ['Good', 'Bad', 'Bad', 'Bad', 'Good', 'Good', 'Good', 'Good','Good','Good']
r = 0
for i in range(1,n):
    working_box = box.copy()
    random.shuffle(working_box)
    sample = list()
    for j in range(1,3):
        sample.append(working_box.pop())
    if (sample[0] == 'Bad' and sample[1] == 'Good'):
        r += 1 
r/n

0.23496